In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('DOGE-USD.csv')

In [3]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,0.000293,0.000299,0.000260,0.000268,0.000268,1.463600e+06
1,2014-09-18,0.000268,0.000325,0.000267,0.000298,0.000298,2.215910e+06
2,2014-09-19,0.000298,0.000307,0.000275,0.000277,0.000277,8.835630e+05
3,2014-09-20,0.000276,0.000310,0.000267,0.000292,0.000292,9.930040e+05
4,2014-09-21,0.000293,0.000299,0.000284,0.000288,0.000288,5.391400e+05
...,...,...,...,...,...,...,...
2496,2021-07-18,0.186841,0.195788,0.180886,0.182233,0.182233,1.424484e+09
2497,2021-07-19,0.181818,0.183950,0.170712,0.173556,0.173556,1.084817e+09
2498,2021-07-20,0.173763,0.178895,0.160728,0.170801,0.170801,1.809479e+09
2499,2021-07-21,0.170527,0.214210,0.168719,0.190349,0.190349,4.433688e+09


In [4]:
df=df[["Close"]].copy()

In [5]:
df

,Close
0,0.000268
1,0.000298
2,0.000277
3,0.000292
4,0.000288
...,...
2496,0.182233
2497,0.173556
2498,0.170801
2499,0.190349


In [6]:
df['target']= df.Close.shift(-1)

In [7]:
df

,Close,target
0,0.000268,0.000298
1,0.000298,0.000277
2,0.000277,0.000292
3,0.000292,0.000288
4,0.000288,0.000298
...,...,...
2496,0.182233,0.173556
2497,0.173556,0.170801
2498,0.170801,0.190349
2499,0.190349,0.190973


In [8]:
df.dropna(inplace=True)

In [9]:
df

,Close,target
0,0.000268,0.000298
1,0.000298,0.000277
2,0.000277,0.000292
3,0.000292,0.000288
4,0.000288,0.000298
...,...,...
2495,0.186125,0.182233
2496,0.182233,0.173556
2497,0.173556,0.170801
2498,0.170801,0.190349


In [10]:
def train_test_split(df, perc):
    df= df.values
    n= int(len(df)* (1-perc))
    return df[:n], df[n:]

In [11]:
train, test = train_test_split(df,0.2)

In [12]:
x= train[:, :-1]
y =train[:, -1]

In [13]:
from lightgbm import LGBMRegressor

In [14]:
ltg =   LGBMRegressor(n_estimators=1000)

In [15]:
ltg .fit(x,y)

LGBMRegressor(n_estimators=1000)

In [16]:
test[2]

array([0.002506, 0.002496])

In [17]:
val =np.array(test[2, 0]).reshape(1, -1)
pred = ltg.predict(val)
pred

array([0.00248425])

# Creating the model in one function


In [28]:
from sklearn.ensemble import GradientBoostingRegressor
def lgt_predict(train, val):
    train = np.array(train)
    x,y = train[:, :-1], train[:, -1]
    #ltg =   LGBMRegressor(n_estimators=1000)
    gre= GradientBoostingRegressor()
    gre.fit(x,y)
    #ltg.fit(x,y)
    val =np.array(val).reshape(1, -1)
    pred = ltg.predict(val)
    return pred[0]

# Walk forward prediction 

In [29]:
from sklearn.metrics import mean_squared_error

def validation(df, perc):
    predictions = []
    train , test = train_test_split(df, perc)
    
    history = [x for x in train]
    
    for i in range(len(test)):
        test_x, test_y = test[i, :-1], test[i -1]
        
        pred = lgt_predict(history, test_x[0])
        predictions.append(pred)
        
        history.append(test[i])
    error = mean_squared_error(test[:,-1], predictions, squared=False)
    return error,  test[:, -1], predictions

In [30]:

ruse, y,pred = validation(df, 0.2)

In [31]:
ruse

0.14950707209194744

In [33]:
len(pred)

499